In [48]:
# https://medium.com/@wachambers/a-beginners-guide-to-building-a-retrieval-augmented-generation-rag-application-from-scratch-e52921953a5d

In [49]:
import requests
import json

In [50]:
corpus_of_documents = [
    "Take a leisurely walk in the park and enjoy the fresh air.",
    "Visit a local museum and discover something new.",
    "Attend a live music concert and feel the rhythm.",
    "Go for a hike and admire the natural scenery.",
    "Have a picnic with friends and share some laughs.",
    "Explore a new cuisine by dining at an ethnic restaurant.",
    "Take a yoga class and stretch your body and mind.",
    "Join a local sports league and enjoy some friendly competition.",
    "Attend a workshop or lecture on a topic you're interested in.",
    "Visit an amusement park and ride the roller coasters."
]

def jaccard_similarity(query, document):
    query = query.lower().split(" ")
    document = document.lower().split(" ")
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

def match_corpus(query, corpus):
  similarities = []
  for doc in corpus:
      similarity = jaccard_similarity(query, doc)
      similarities.append(similarity)
  return corpus_of_documents[similarities.index(max(similarities))]


In [51]:
user_prompt = "What is a leisure activity that you like?"
user_input = "I love a nice picnic"

match_corpus(user_input, corpus_of_documents)

'Have a picnic with friends and share some laughs.'

# Feed results to llama2

In [52]:
user_input = "I like to hike"
relevant_document = match_corpus(user_input, corpus_of_documents)
full_response = []

In [53]:
prompt = """
You are a bot that makes recommendations for activities. You answer in very short sentences and do not include extra information.
This is the recommended activity: {relevant_document}
The user input is: {user_input}
Compile a recommendation to the user based on the recommended activity and the user input.
"""

In [54]:
url = 'http://localhost:11434/api/generate'
data = {
    "model": "llama2",
    "prompt": prompt.format(user_input=user_input, relevant_document=relevant_document)
}

In [55]:
headers = {'Content-Type': 'application/json'}
response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)
try:
    count = 0
    for line in response.iter_lines():
        # filter out keep-alive new lines
        # count += 1
        # if count % 5== 0:
        #     print(decoded_line['response']) # print every fifth token
        if line:
            decoded_line = json.loads(line.decode('utf-8'))

            full_response.append(decoded_line['response'])
finally:
    response.close()
print(''.join(full_response))

 Great! Based on your input, I recommend going for a hike in a nearby nature reserve. The scenery will be absolutely breathtaking and you'll get some exercise too!


# Bring it together 

In [56]:
def fetch_llama_response(user_input):
  relevant_document = match_corpus(user_input, corpus_of_documents)

  full_response = []
  prompt = """
You are a bot that makes recommendations for activities. You answer in very short sentences and do not include extra information.
This is the recommended activity: {relevant_document}
The user input is: {user_input}
Compile a recommendation to the user based on the recommended activity and the user input.
""".format(user_input=user_input, relevant_document=relevant_document)

  url = 'http://localhost:11434/api/generate'
  data = {
      "model": "llama2",
      "prompt": prompt
  }

  headers = {'Content-Type': 'application/json'}
  response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)
  try:
      count = 0
      for line in response.iter_lines():
          # filter out keep-alive new lines
          # count += 1
          # if count % 5== 0:
          #     print(decoded_line['response']) # print every fifth token
          if line:
              decoded_line = json.loads(line.decode('utf-8'))

              full_response.append(decoded_line['response'])
  finally:
      response.close()
  return ''.join(full_response)


In [58]:
user_input = input("What is a leisure activity that you like?")

fetch_llama_response(user_input)

' Sure, I\'d be happy to recommend an activity! Based on your input of "theater," I suggest you try... watching a movie at home! Enjoy the fresh air and relaxation from the comfort of your own space.'